<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/Bank_Doc_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.3 MB/s eta 0:00:00


In [1]:
!pip install PyPDF2
!pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [5]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (642 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
# 1. Access Google Drive

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


# LINEAR REGRESSION MODEL

In [11]:
import PyPDF2
import os

def pdf_to_text(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = " ".join([page.extract_text() for page in reader.pages])
    return text

# 3. Prepare data for training

from sklearn.feature_extraction.text import TfidfVectorizer

# Path to the main directory containing the subfolders of 'AC' and 'BS'
ground_truth_dir = '/content/gdrive/MyDrive/BANK UC'

# Define subfolder names
clusters_names = ['AC', 'Statements']

# Fetch documents from the folders
documents = []
labels = []

for idx, cluster in enumerate(clusters_names):
    cluster_path = os.path.join(ground_truth_dir, cluster)
    pdf_files = [f for f in os.listdir(cluster_path) if f.endswith('.pdf')]
    for pdf_file in pdf_files:
        documents.append(os.path.join(cluster_path, pdf_file))
        labels.append(idx)


texts = [pdf_to_text(doc) for doc in documents]

vectorizer = TfidfVectorizer(max_features=5000)
X_text = vectorizer.fit_transform(texts)

# 4. Split data

from sklearn.model_selection import train_test_split

X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(X_text, labels, test_size=0.2, random_state=42)

# 5. Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_model = LogisticRegression()
lr_model.fit(X_train_text, y_train_text)

lr_predictions = lr_model.predict(X_test_text)
lr_accuracy = accuracy_score(y_test_text, lr_predictions)

# 6. Output scores

print(f"Logistic Regression Accuracy: {lr_accuracy}")


Logistic Regression Accuracy: 1.0


## K Folds Cross Validation

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming you already have the texts and labels from the previous code:

# Convert texts to feature vectors
vectorizer = TfidfVectorizer(max_features=5000)
X_text = vectorizer.fit_transform(texts)

# Initialize the Logistic Regression model
lr_model = LogisticRegression()

# Apply k-fold cross-validation
k = 5
scores = cross_val_score(lr_model, X_text, labels, cv=k, scoring='accuracy')

# Print out the scores
for fold, score in enumerate(scores, 1):
    print(f"Fold {fold}: Accuracy: {score:.4f}")

# Print the average score
print(f"\nAverage Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")


Fold 1: Accuracy: 1.0000
Fold 2: Accuracy: 1.0000
Fold 3: Accuracy: 1.0000
Fold 4: Accuracy: 1.0000
Fold 5: Accuracy: 1.0000

Average Accuracy: 1.0000 ± 0.0000


# CNN MODEL

In [17]:
# Imports and Setup

import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from pdf2image import convert_from_path
from google.colab import drive

drive.mount('/content/gdrive')

# Convert PDFs to Images (Only First Page)

def pdf_to_image(pdf_path):
    # Extract only the first page
    return convert_from_path(pdf_path, first_page=1, last_page=1)[0]

def process_images_from_pdfs(paths, labels):
    data = []
    new_labels = []
    for idx, path in enumerate(paths):
        img = pdf_to_image(path)
        img = img.resize((128, 128))
        data.append(np.array(img))
        new_labels.append(labels[idx])
    return np.array(data), new_labels

# Paths and Labels

folder_path = '/content/gdrive/MyDrive/Classification'

documents = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]

labels = []
for doc in documents:
    if "AC" in os.path.basename(doc):
        labels.append(0)
    elif "file" in os.path.basename(doc):
        labels.append(1)
    else:
        print(f"Cannot assign label for file: {os.path.basename(doc)}")
        documents.remove(doc)

img_data, labels = process_images_from_pdfs(documents, labels)
X_img = img_data / 255.0  # Normalize image data

# Convert labels to numpy array
labels = np.array(labels)

# Split Data

X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(X_img, labels, test_size=0.2, random_state=42)

# Build, Train, and Evaluate CNN

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Assuming only two categories: AC and Statements
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_img, y_train_img, epochs=5)

cnn_accuracy = model.evaluate(X_test_img, y_test_img, verbose=2)[1]

# Output Scores

print(f"CNN Accuracy: {cnn_accuracy}")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 0.5900 - accuracy: 0.7391
Epoch 2/5
1/1 [==============================] - 0s 384ms/step - loss: 13.1661 - accuracy: 0.7391
Epoch 3/5
1/1 [==============================] - 0s 363ms/step - loss: 8.4398 - accuracy: 0.2609
Epoch 4/5
1/1 [==============================] - 0s 354ms/step - loss: 3.3591 - accuracy: 0.7391
Epoch 5/5
1/1 [==============================] - 0s 334ms/step - loss: 5.2034 - accuracy: 0.7391
1/1 - 0s - loss: 4.2651 - accuracy: 0.6667 - 225ms/epoch - 225ms/step
CNN Accuracy: 0.6666666865348816


## **CNN Accuracy: 0.6666666865348816**